In [147]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import bs4 as bs
import urllib.request
import re
import nltk
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
#from nltk.corpus import 
%matplotlib inline

In [48]:
nltk.download('wordnet')

[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [148]:
#nltk.download('stopwords')
stop = stopwords.words('english')

In [149]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Aditya
[nltk_data]     Rane\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [150]:
data_path = "D:/Data_Science/Recommender systems/the-movies-dataset/movies_metadata.csv"
#data_path=data_path.replace("\\","/")
print(data_path)
                        

D:/Data_Science/Recommender systems/the-movies-dataset/movies_metadata.csv


In [151]:
movies_metadata = pd.read_csv(data_path,low_memory=False)
movies_metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [152]:
movies_metadata.shape

(45466, 24)

In [153]:
vote_average = movies_metadata.vote_average.mean()
print(vote_average)

5.618207215133889


In [154]:
m_90 = movies_metadata.vote_count.quantile(0.90)
print(m_90)

160.0


In [155]:
q_movies = movies_metadata.copy().loc[movies_metadata['vote_count'] >= m_90]
print(q_movies.shape)
q_movies.head()

(4555, 24)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
8,False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,1995-12-22,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0


In [156]:
q_movies.shape

(4555, 24)

In [157]:
def weighted_rating(x, m_90 = m_90 , vote_average = vote_average):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m_90) * R) + (m_90/(m_90+v) * vote_average)


In [158]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)
q_movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,score
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,7.640253
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,6.820293
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,5.660700
5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,7.537201
8,False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0,5.556626


In [159]:
#Sort movies based on score calculated above
q_movies = q_movies.sort_values('score', ascending=False)
q_movies = q_movies.reset_index()
#Print the top 15 movies
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(15)

,title,vote_count,vote_average,score
0,The Shawshank Redemption,8358.0,8.5,8.445869
1,The Godfather,6024.0,8.5,8.425439
2,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453
3,The Dark Knight,12269.0,8.3,8.265477
4,Fight Club,9678.0,8.3,8.256385
5,Pulp Fiction,8670.0,8.3,8.251406
6,Schindler's List,4436.0,8.3,8.206639
7,Whiplash,4376.0,8.3,8.205404
8,Spirited Away,3968.0,8.3,8.196055
9,Life Is Beautiful,3643.0,8.3,8.187171


In [160]:
q_movies['overview'].head()

0    Framed in the 1940s for the double murder of h...
1    Spanning the years 1945 to 1955, a chronicle o...
2    Raj is a rich, carefree, happy-go-lucky second...
3    Batman raises the stakes in his war on crime. ...
4    A ticking-time-bomb insomniac and a slippery s...
Name: overview, dtype: object

In [161]:
#cleaning data
q_movies['overview'] = q_movies['overview'].fillna('')
q_movies['overview'] = q_movies['overview'].str.lower().replace('[^\w\s]','')
q_movies['overview'] = q_movies['overview'].str.replace('\d+','')
q_movies['overview'] = q_movies['overview'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [162]:
q_movies['overview'].str.split()

0       [framed, double, murder, wife, lover,, upstand...
1       [spanning, years, ,, chronicle, fictional, ita...
2       [raj, rich,, carefree,, happy-go-lucky, second...
3       [batman, raises, stakes, war, crime., help, lt...
4       [ticking-time-bomb, insomniac, slippery, soap,...
5       [burger-loving, hit, man,, philosophical, part...
6       [true, story, businessman, oskar, schindler, s...
7       [direction, ruthless, instructor,, talented, y...
8       [ten, year, old, girl, wanders, away, parents,...
9       [touching, story, italian, book, seller, jewis...
10      [continuing, saga, corleone, crime, family,, y...
11      [serving, time, insanity, state, mental, hospi...
12      [man, low, iq, accomplished, great, things, li...
13      [epic, saga, continues, luke, skywalker,, hope...
14      [larcenous, real, estate, clerk, marion, crane...
15      [true, story, two, men, never, met, –, quadrip...
16      [high, schoolers, mitsuha, taki, complete, str...
17      [leon,

In [163]:
q_movies['overview'][939]

''

In [164]:
#Frequent word removal
freq = pd.Series(' '.join(q_movies['overview']).split()).value_counts()[:10]
freq = list(freq.index)
q_movies['overview'] = q_movies['overview'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
q_movies['overview'].head()

0    framed double murder wife lover, upstanding ba...
1    spanning years , chronicle fictional italian-a...
2    raj rich, carefree, happy-go-lucky second gene...
3    batman raises stakes war crime. help lt. jim g...
4    ticking-time-bomb insomniac slippery soap sale...
Name: overview, dtype: object

In [165]:
#Rare word removal
freq = pd.Series(' '.join(q_movies['overview']).split()).value_counts()[-10:]
freq = list(freq.index)
q_movies['overview'] = q_movies['overview'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
q_movies['overview'].head()

0    framed double murder wife lover, upstanding ba...
1    spanning years , chronicle fictional italian-a...
2    raj rich, carefree, happy-go-lucky second gene...
3    batman raises stakes war crime. help lt. jim g...
4    ticking-time-bomb insomniac slippery soap sale...
Name: overview, dtype: object

In [166]:
from nltk.stem import PorterStemmer
st = PorterStemmer()
q_movies['overview'] = q_movies['overview'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
q_movies['overview'].head()

0    frame doubl murder wife lover, upstand banker ...
1    span year , chronicl fiction italian-american ...
2    raj rich, carefree, happy-go-lucki second gene...
3    batman rais stake war crime. help lt. jim gord...
4    ticking-time-bomb insomniac slipperi soap sale...
Name: overview, dtype: object

In [167]:
from textblob import Word
q_movies['overview'] = q_movies['overview'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
q_movies['overview'].head()

0    frame doubl murder wife lover, upstand banker ...
1    span year , chronicl fiction italian-american ...
2    raj rich, carefree, happy-go-lucki second gene...
3    batman rais stake war crime. help lt. jim gord...
4    ticking-time-bomb insomniac slipperi soap sale...
Name: overview, dtype: object

In [168]:
sentence_list = []
for i in q_movies['overview']:
    sentence_list.append(i)
    #break
sentence_list

['frame doubl murder wife lover, upstand banker andi dufresn begin shawshank prison, put account skill work amor warden. long stretch prison, dufresn come admir inmat -- includ older prison name red -- integr unquench sen hope.',
 'span year , chronicl fiction italian-american corleon crime family. organ crime famili patriarch, vito corleon bare surviv attempt life, youngest son, michael step take care would-b killers, launch campaign bloodi revenge.',
 'raj rich, carefree, happy-go-lucki second gener nri. simran daughter chaudhari baldev singh, spite nri strict adher indian values. simran left india marri childhood fiancé. raj leav india mission hands, claim ladi love nose whole family. thu begin saga.',
 'batman rais stake war crime. help lt. jim gordon district attorney harvey dent, batman set dismantl remain crimin organ plagu streets. partnership prove effective, soon prey reign chao unleash rise crimin mastermind known terrifi citizen gotham joker.',
 'ticking-time-bomb insomniac

In [169]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(sentence_list)]

In [170]:
max_epochs = 100
vec_size = 128
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

In [171]:
for epoch in range(max_epochs):
    if epoch % 10 == 0 :
        print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

iteration 0
iteration 10
iteration 20
iteration 30
iteration 40
iteration 50
iteration 60
iteration 70
iteration 80
iteration 90
Model Saved


In [172]:
def get_recommendation(title):
    test_data = word_tokenize(title.lower())
    print('ORIGINAL TEXT : {}'.format(title))
    v1 = model.infer_vector(test_data)
    similar_doc = model.docvecs.most_similar([v1])
    print('RECOMMENDED MOVIES :')
    num = 1
    for i in similar_doc:
        idx = int(i[0])
        description = q_movies['overview'].iloc[idx]
        original_title = q_movies['original_title'].iloc[idx]
        print('recommendation {} : {}\ndescription : {}\n'.format(num,original_title,description))
        num+=1
        #print(idx)

In [173]:
title = 'mysterious story two magicians whose intense rivalry leads lifelong battle supremacy full obsession deceit jealousy dangerous deadly consequences'
get_recommendation(title)

ORIGINAL TEXT : mysterious story two magicians whose intense rivalry leads lifelong battle supremacy full obsession deceit jealousy dangerous deadly consequences
RECOMMENDED MOVIES :
recommendation 1 : Il ricco, il povero e il maggiordomo
description : 

recommendation 2 : Chiedimi se sono felice
description : 

recommendation 3 : Cado dalle nubi
description : 

recommendation 4 : La Banda Dei Babbi Natale
description : 

recommendation 5 : Tu la conosci Claudia?
description : 

recommendation 6 : King Kong
description : adventur film film crew search monster remot island. crew king kong decid take back york money make spectacle. film masterpiec stop-mot filmmak histori inspir line king kong films.

recommendation 7 : Race
description : base incred true stori jess owens, legendari athlet superstar whose quest becom greatest track field athlet histori thrust onto stage olympics, face adolf hitler' vision aryan supremacy. star stephan james, jason sudeikis, jeremi irons, caric van houten

In [91]:
test_data = word_tokenize("framed double murder wife lover upstanding banker andy dufresne begins new life shawshank prison puts accounting skills work amoral warden long stretch prison dufresne comes admired inmates including older prisoner named red integrity unquenchable sense hope".lower())
v1 = model.infer_vector(test_data)
v1

array([ 0.06723744, -0.24525112,  0.01416143,  0.4558571 , -0.4983364 ,
        0.31983948,  0.12748107, -0.0379651 ,  0.12071222, -0.33105662,
       -0.3037436 ,  0.2799338 , -0.5472637 , -0.05302281, -0.20357902,
       -0.00990565, -0.51407593, -0.0309389 , -0.25431263, -0.575303  ],
      dtype=float32)

In [97]:
test_data

['framed',
 'double',
 'murder',
 'wife',
 'lover',
 'upstanding',
 'banker',
 'andy',
 'dufresne',
 'begins',
 'new',
 'life',
 'shawshank',
 'prison',
 'puts',
 'accounting',
 'skills',
 'work',
 'amoral',
 'warden',
 'long',
 'stretch',
 'prison',
 'dufresne',
 'comes',
 'admired',
 'inmates',
 'including',
 'older',
 'prisoner',
 'named',
 'red',
 'integrity',
 'unquenchable',
 'sense',
 'hope']

In [101]:
similar_doc = model.docvecs.most_similar([v1])
print(similar_doc)

[('1997', 0.7643126249313354), ('1998', 0.7540237903594971), ('1134', 0.7371810674667358), ('2440', 0.730514645576477), ('4014', 0.7224717140197754), ('3191', 0.7012327313423157), ('2664', 0.6994910836219788), ('37', 0.6967129111289978), ('951', 0.6956272125244141), ('1636', 0.6944336891174316)]


In [102]:
for i in similar_doc:
    idx = i[0]
    print(idx)

1997
1998
1134
2440
4014
3191
2664
37
951
1636


In [107]:
tagged_data[1997]

TaggedDocument(words=['holidaying', 'french', 'alps', 'swedish', 'family', 'deals', 'acts', 'cowardliness', 'avalanche', 'breaks'], tags=['1997'])

In [109]:
q_movies['overview'].iloc[0]

'framed double murder wife lover upstanding banker andy dufresne begins new life shawshank prison puts accounting skills work amoral warden long stretch prison dufresne comes admired inmates including older prisoner named red integrity unquenchable sense hope'

In [111]:
q_movies['overview'].iloc[1636]

'twentytwo years events jurassic park isla nublar features fully functioning dinosaur theme park jurassic world originally envisioned john hammond'

In [76]:
all_words = [nltk.word_tokenize(sent) for sent in q_movies['overview']]

In [79]:
word2vec = Word2Vec(all_words, min_count=3)

In [80]:
vocabulary = word2vec.wv.vocab
print(vocabulary)

{'frame': <gensim.models.keyedvectors.Vocab object at 0x000002D3DA90D9B0>, 'doubl': <gensim.models.keyedvectors.Vocab object at 0x000002D3DA90D940>, 'murder': <gensim.models.keyedvectors.Vocab object at 0x000002D3DA90D908>, 'wife': <gensim.models.keyedvectors.Vocab object at 0x000002D3DA90D6A0>, 'lover': <gensim.models.keyedvectors.Vocab object at 0x000002D3DA90DBE0>, 'banker': <gensim.models.keyedvectors.Vocab object at 0x000002D3DA90D5F8>, 'andi': <gensim.models.keyedvectors.Vocab object at 0x000002D3DA90D748>, 'begin': <gensim.models.keyedvectors.Vocab object at 0x000002D3DA90D860>, 'new': <gensim.models.keyedvectors.Vocab object at 0x000002D3DA90D8D0>, 'prison': <gensim.models.keyedvectors.Vocab object at 0x000002D3DA90DCF8>, 'put': <gensim.models.keyedvectors.Vocab object at 0x000002D3DA90D7F0>, 'account': <gensim.models.keyedvectors.Vocab object at 0x000002D3DA90DDD8>, 'skill': <gensim.models.keyedvectors.Vocab object at 0x000002D3DA90D780>, 'work': <gensim.models.keyedvectors.Vo

In [84]:
v1 = word2vec.wv['skill']
len(v1)

100

In [92]:
sim_words = word2vec.wv.most_similar('queen')
sim_words

[('use', 0.9999587535858154),
 ('hous', 0.9999514222145081),
 ('Dr', 0.9999498128890991),
 ('final', 0.9999492764472961),
 ('leav', 0.9999479055404663),
 ('land', 0.9999479055404663),
 ('mission', 0.9999467134475708),
 ('she', 0.999946653842926),
 ('lead', 0.9999462962150574),
 ('he', 0.999946117401123)]

In [93]:
v1 = word2vec.wv['king']
v2 = word2vec.wv['man']
v3 = word2vec.wv['woman']
v4 = v1-v2+v3

In [97]:
word2vec.wv['queen']

array([-3.62991989e-01,  6.63586259e-02, -3.81433487e-01, -1.86369792e-01,
        7.86567748e-01,  9.12404153e-03, -1.76669117e-02, -5.98441437e-02,
        8.20351988e-02,  9.95816737e-02,  1.64690018e-02,  3.41257602e-01,
       -4.78362620e-01, -3.68348435e-02,  1.72220916e-01,  1.25623357e+00,
        1.87752202e-01,  1.00088432e-01,  2.73932214e-03,  4.33194697e-01,
       -1.54756457e-01, -1.99885920e-01,  1.55705558e-02, -1.32024437e-01,
       -9.91931185e-02,  8.28435123e-02, -1.70661077e-01, -9.99640524e-02,
        7.00688586e-02, -1.50580108e-01,  7.54732639e-03, -1.96074411e-01,
       -2.14914680e-02,  3.58185351e-01, -6.38156712e-01, -2.27705836e-01,
        4.22754318e-01,  3.08776557e-01, -5.63351095e-01, -3.75720680e-01,
        5.31016700e-02,  3.71474236e-01,  3.46192449e-01, -2.57272750e-01,
        7.60311604e-01,  2.63873905e-01, -9.45933342e-01,  3.39460224e-01,
       -1.37021899e-01, -8.07186544e-01,  9.44444090e-02,  1.93670305e-04,
       -4.25136328e-01, -

In [96]:
word2vec.wv.similar_by_vector(v4)

[('king', 0.9998806715011597),
 ('woman', 0.9998610019683838),
 ('magic', 0.9998524785041809),
 ('Dr', 0.9998456835746765),
 ('howev', 0.9998446106910706),
 ('happen', 0.999841570854187),
 ('compani', 0.9998409748077393),
 ('book', 0.9998376965522766),
 ('escap', 0.9998370409011841),
 ('stranger', 0.9998366832733154)]

In [98]:
v1.shape

(100,)